In [ ]:
%pip install tensorflow
%pip install numpy
%pip install pillow

import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Input, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle

train_data_dir = r"C:\Users\asims\Desktop\hackathon\archive (1)\train"
test_data_dir = r"C:\Users\asims\Desktop\hackathon\archive (1)\test"
input_shape = (224, 224, 1)

batch_size = 32
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical')

cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
cnn_model.add(MaxPooling2D((2, 2))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2))
cnn_model.add(Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2))
cnn_model.add(Flatten())

lstm_input = Input(shape=(timesteps,) + input_shape)
cnn_features = TimeDistributed(cnn_model)(lstm_input)
lstm_layer = LSTM(3)(cnn_features)  # 3 is the number of output classes

output_layer = Dense(3, activation='softmax')(lstm_layer)

classification_model = Model(inputs=lstm_input, outputs=output_layer)

classification_model.compile(loss='categorical_crossentropy',
                              optimizer=Adam(lr=0.0001),
                              metrics=['accuracy'])

history = classification_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=10)
